<a href="https://colab.research.google.com/github/cyberXsaurabh/Symptom-classification-bot/blob/main/simple_langgraph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# STEP 0: Install dependencies

In [1]:
!pip install -qU google-generativeai==0.8.5 google-ai-generativelanguage==0.6.15 \
langgraph langchain langchain-google-genai openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.6/755.6 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 15.7 MB/s eta 0:00:00


# STEP 1: Imports and secure API key input

In [2]:
import os
import getpass
from langgraph.graph import StateGraph, END
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage


# Secure Gemini API Key input

In [3]:
os.environ['GOOGLE_API_KEY'] = getpass.getpass("Enter the Gemini API Key")


Enter the Gemini API Key··········


# STEP 2: Initialize Gemini 1.5 Flash

In [4]:
llm = ChatGoogleGenerativeAI(model = "models/gemini-1.5-flash-latest", temperature = 0.2)

# STEP 3: Node to ask user for symptom

In [5]:
def get_symptom(state: dict) ->dict:
  symptom= input("Welcome to hospital, Please Enter your sympotom:")
  state["symptom"]=symptom
  return state

# STEP 4: Node to classify the symptom

In [6]:
def classify_symptom(state:dict) ->dict:
  prompt=(
      "You are a helpful Medical Assistant, Classify the symptoms below into one of the categories \n"
      "-General\n -Emergency \n -Mental Health\n"
      f"Symptom : {state['symptom']}\n"
      "Respond only with onew word : General, Emergency or Mental Health"
      "Example : input : I have fever, Output : General"
  )
  response = llm.invoke([HumanMessage(content=prompt)])
  category = response.content.strip()
  print(f"LLM classifies the symptom as: {category}") #debug
  state["category"]=category
  return state

# STEP 5: Router logic to route to the correct node


In [20]:
def symptom_router(state:dict) -> dict:
  cat = state["category"].lower()
  if "general" in cat:
    return "general"
  elif "emergency"in cat:
    return "emergency"
  elif "mental" in cat:
    return "mental_health"
  else:
    return "general"


# STEP 6: Category-specific response nodes


In [21]:
def general_node(state:dict) -> dict:
  state["answer"] = f" '{state['symptom']}' seems general : directing you to general ward for consulting a doctor"
  return state

def emergency_node(state:dict) -> dict:
  state["answer"] = f" '{state['symptom']}' : It is a medical emergency : seeking immediate help"
  return state

def mental_health_node(state:dict) -> dict:
  state["answer"] = f" '{state['symptom']}'  Seems like a medical health issue : talk to our counsellor"
  return state





# STEP 7: Build LangGraph


In [22]:
builder = StateGraph(dict)

builder.set_entry_point("get_symptom")

builder.add_node("get_symptom", get_symptom)
builder.add_node("classify_symptom", classify_symptom)
builder.add_node("general", general_node)
builder.add_node("emergency", emergency_node)
builder.add_node("mental_health", mental_health_node)

builder.add_edge("get_symptom", "classify_symptom")

builder.add_conditional_edges(
    "classify_symptom",
    symptom_router,  # this should be a function returning a key like 'general', 'emergency', etc.
    {
        "general": "general",
        "emergency": "emergency",
        "mental_health": "mental_health"
    }
)

builder.add_edge("general", END)
builder.add_edge("emergency", END)
builder.add_edge("mental_health", END)


# STEP 8: Compile and invoke the graph


In [23]:
graph = builder.compile()


In [27]:
final_state = graph.invoke({})
print("final output\n")
print(final_state["answer"])

Welcome to hospital, Please Enter your sympotom:fever
LLM classifies the symptom as: General
final output

 'fever' seems general : directing you to general ward for consulting a doctor
